In [1]:
import tkinter as tk
from tkinter import ttk, messagebox, scrolledtext
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
import requests
import json
import re
from datetime import datetime

class ShoppingListApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Smart Shopping List App")
        self.root.geometry("800x900")
        
        # Perplexity API configuration
        self.api_key = "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"
        self.api_url = "https://api.perplexity.ai/chat/completions"
        self.headers = {
            "Authorization": f"Bearer {self.api_key}",
            "Content-Type": "application/json"
        }
        
        # Shopping list storage
        self.shopping_items = []
        self.max_items = 10
        
        # Supermarket brand mappings
        self.supermarket_brands = ["Tesco", "Sainsbury", "Morrison", "Waitrose", "Co-op", "Coop", "Asda"]
        self.website_mapping = {
            "Tesco": "www.tesco.com",
            "Sainsbury": "www.sainsburys.co.uk", 
            "Morrison": "www.morrisons.com",
            "Waitrose": "www.waitrose.com",
            "Co-op": "www.coop.co.uk",
            "Coop": "www.coop.co.uk",
            "Asda": "www.asda.com"
        }
        
        # Gmail credentials
        self.gmail_email = "onsalaprojects@gmail.com"
        self.gmail_password = "udgvitnjpzycbjwz"
        
        self.create_widgets()
    
    def create_widgets(self):
        # Main frame with scrollbar
        main_frame = ttk.Frame(self.root)
        main_frame.pack(fill=tk.BOTH, expand=True, padx=10, pady=10)
        
        # Title
        title_label = ttk.Label(main_frame, text="Smart Shopping List App", font=("Arial", 16, "bold"))
        title_label.pack(pady=(0, 20))
        
        # Shopping list section
        shopping_frame = ttk.LabelFrame(main_frame, text="Shopping List (Max 10 items)", padding=10)
        shopping_frame.pack(fill=tk.X, pady=(0, 10))
        
        # Item entry frame
        self.item_frame = ttk.Frame(shopping_frame)
        self.item_frame.pack(fill=tk.X)
        
        # Headers
        ttk.Label(self.item_frame, text="Item Name", font=("Arial", 10, "bold")).grid(row=0, column=0, padx=5, pady=5, sticky="w")
        ttk.Label(self.item_frame, text="Quantity", font=("Arial", 10, "bold")).grid(row=0, column=1, padx=5, pady=5, sticky="w")
        
        # Initial item row
        self.add_item_row()
        
        # Add item button
        add_button = ttk.Button(shopping_frame, text="Add Item", command=self.add_item_row)
        add_button.pack(pady=10)
        
        # User details section
        details_frame = ttk.LabelFrame(main_frame, text="Your Details", padding=10)
        details_frame.pack(fill=tk.X, pady=(0, 10))
        
        # Postcode
        ttk.Label(details_frame, text="British Postcode:").grid(row=0, column=0, sticky="w", padx=5, pady=5)
        self.postcode_entry = ttk.Entry(details_frame, width=20)
        self.postcode_entry.grid(row=0, column=1, padx=5, pady=5, sticky="w")
        
        # First name
        ttk.Label(details_frame, text="First Name:").grid(row=1, column=0, sticky="w", padx=5, pady=5)
        self.firstname_entry = ttk.Entry(details_frame, width=30)
        self.firstname_entry.grid(row=1, column=1, padx=5, pady=5, sticky="w")
        
        # Email
        ttk.Label(details_frame, text="Email Address:").grid(row=2, column=0, sticky="w", padx=5, pady=5)
        self.email_entry = ttk.Entry(details_frame, width=40)
        self.email_entry.grid(row=2, column=1, padx=5, pady=5, sticky="w")
        
        # Process button
        process_button = ttk.Button(main_frame, text="Process Shopping List", command=self.process_shopping_list)
        process_button.pack(pady=20)
        
        # Results display
        self.results_text = scrolledtext.ScrolledText(main_frame, height=15, width=80)
        self.results_text.pack(fill=tk.BOTH, expand=True, pady=(10, 0))
    
    def add_item_row(self):
        if len(self.shopping_items) >= self.max_items:
            messagebox.showwarning("Limit Reached", f"Maximum {self.max_items} items allowed")
            return
        
        row = len(self.shopping_items) + 1
        
        # Item name entry
        name_entry = ttk.Entry(self.item_frame, width=30)
        name_entry.grid(row=row, column=0, padx=5, pady=2, sticky="w")
        
        # Quantity entry
        quantity_entry = ttk.Entry(self.item_frame, width=10)
        quantity_entry.grid(row=row, column=1, padx=5, pady=2, sticky="w")
        
        # Remove button
        remove_button = ttk.Button(self.item_frame, text="Remove", 
                                 command=lambda idx=len(self.shopping_items): self.remove_item_row(idx))
        remove_button.grid(row=row, column=2, padx=5, pady=2)
        
        self.shopping_items.append({
            'name_entry': name_entry,
            'quantity_entry': quantity_entry,
            'remove_button': remove_button
        })
    
    def remove_item_row(self, index):
        if len(self.shopping_items) <= 1:
            messagebox.showwarning("Cannot Remove", "At least one item is required")
            return
        
        # Remove widgets
        item = self.shopping_items[index]
        item['name_entry'].destroy()
        item['quantity_entry'].destroy()
        item['remove_button'].destroy()
        
        # Remove from list
        self.shopping_items.pop(index)
        
        # Refresh the display
        self.refresh_item_display()
    
    def refresh_item_display(self):
        # Clear and recreate all item rows
        for widget in self.item_frame.winfo_children():
            if int(widget.grid_info()["row"]) > 0:
                widget.destroy()
        
        items_data = []
        for item in self.shopping_items:
            name = item['name_entry'].get()
            quantity = item['quantity_entry'].get()
            items_data.append((name, quantity))
        
        self.shopping_items.clear()
        
        # Recreate headers
        ttk.Label(self.item_frame, text="Item Name", font=("Arial", 10, "bold")).grid(row=0, column=0, padx=5, pady=5, sticky="w")
        ttk.Label(self.item_frame, text="Quantity", font=("Arial", 10, "bold")).grid(row=0, column=1, padx=5, pady=5, sticky="w")
        
        # Recreate rows with preserved data
        for name, quantity in items_data:
            self.add_item_row()
            if name:
                self.shopping_items[-1]['name_entry'].insert(0, name)
            if quantity:
                self.shopping_items[-1]['quantity_entry'].insert(0, quantity)
    
    def get_shopping_list(self):
        items = []
        for i, item in enumerate(self.shopping_items):
            name = item['name_entry'].get().strip()
            quantity = item['quantity_entry'].get().strip()
            
            if name and quantity:
                try:
                    qty = int(quantity)
                    items.append({
                        'item_id': f"Item_{i+1}",
                        'name': name,
                        'quantity': qty
                    })
                except ValueError:
                    messagebox.showerror("Invalid Input", f"Quantity for '{name}' must be a number")
                    return None
        
        if not items:
            messagebox.showerror("No Items", "Please add at least one item to your shopping list")
            return None
        
        return items
    
    def validate_inputs(self):
        postcode = self.postcode_entry.get().strip()
        firstname = self.firstname_entry.get().strip()
        email = self.email_entry.get().strip()
        
        if not postcode:
            messagebox.showerror("Missing Input", "Please enter a British postcode")
            return False
        
        if not firstname:
            messagebox.showerror("Missing Input", "Please enter your first name")
            return False
        
        if not email or '@' not in email:
            messagebox.showerror("Invalid Email", "Please enter a valid email address")
            return False
        
        return True
    
    def query_perplexity_sonar(self, prompt, system_message="You are a helpful assistant."):
        """Query Perplexity Sonar API directly without OpenAI references"""
        try:
            payload = {
                "model": "sonar-pro",
                "messages": [
                    {"role": "system", "content": system_message},
                    {"role": "user", "content": prompt}
                ],
                "max_tokens": 2000,
                "temperature": 0.3,
                "top_p": 0.9,
                "search_domain_filter": ["perplexity.ai"],
                "return_images": False,
                "return_related_questions": False,
                "search_recency_filter": "month",
                "top_k": 0,
                "stream": False,
                "presence_penalty": 0,
                "frequency_penalty": 1
            }
            
            response = requests.post(self.api_url, headers=self.headers, json=payload)
            
            if response.status_code == 200:
                result = response.json()
                return result['choices'][0]['message']['content']
            else:
                self.results_text.insert(tk.END, f"API Error: {response.status_code} - {response.text}\n")
                return None
                
        except Exception as e:
            self.results_text.insert(tk.END, f"API Connection Error: {str(e)}\n")
            return None
    
    def find_closest_supermarket(self, postcode):
        brands_str = ", ".join(self.supermarket_brands)
        prompt = f"""
        Find the closest supermarket to postcode {postcode} in the UK that matches one of these brands: {brands_str}.
        
        Please provide the response in this exact format:
        Supermarket Name: [exact name]
        Brand: [matching brand from the list]
        Address: [full postal address]
        Distance: [distance in km]
        Driving Time: [time in minutes]
        Opening Hours Today: [today's opening hours for Sunday, May 25, 2025]
        
        Use current real-time data and Google Maps for distance and driving time calculations.
        """
        
        return self.query_perplexity_sonar(prompt, "You are a location search assistant. Provide accurate, real-time information about UK supermarkets.")
    
    def get_item_prices(self, items, supermarket_brand):
        website = self.website_mapping.get(supermarket_brand, "www.tesco.com")
        
        prices = {}
        for item in items:
            prompt = f"""
            Search for the current price of "{item['name']}" on {website}.
            
            Please provide only the numerical price in pounds (£) without the currency symbol.
            If multiple options exist, provide the lowest price available.
            
            Format your response as: [numerical price only]
            """
            
            response = self.query_perplexity_sonar(prompt, f"You are a price comparison assistant. Search {website} for current product prices.")
            
            if response:
                # Extract numerical price
                price_match = re.search(r'[\d.]+', response)
                if price_match:
                    try:
                        prices[item['item_id']] = float(price_match.group())
                    except ValueError:
                        prices[item['item_id']] = 0.0
                else:
                    prices[item['item_id']] = 0.0
            else:
                prices[item['item_id']] = 0.0
        
        return prices
    
    def sense_check_prices(self, items, prices):
        items_price_str = ""
        for item in items:
            price = prices.get(item['item_id'], 0.0)
            items_price_str += f"{item['name']}: £{price}\n"
        
        prompt = f"""
        Please sense check these prices for UK supermarket items:
        
        {items_price_str}
        
        For any price that looks abnormal or unreasonable, please substitute it with the lowest price you can find on tesco.com for that specific item.
        
        Please provide the corrected prices in this exact format:
        [Item Name]: [Corrected Price] [UPDATED if changed, ORIGINAL if unchanged]
        
        Ask yourself - does each price look reasonable for a UK supermarket?
        """
        
        response = self.query_perplexity_sonar(prompt, "You are a price validation assistant. Verify UK supermarket prices and suggest corrections.")
        
        # Parse the response to update prices
        updated_prices = prices.copy()
        updated_flags = {}
        
        if response:
            lines = response.split('\n')
            for line in lines:
                if ':' in line and ('£' in line or any(char.isdigit() for char in line)):
                    parts = line.split(':')
                    if len(parts) >= 2:
                        item_name = parts[0].strip()
                        price_part = parts[1].strip()
                        
                        # Find matching item
                        for item in items:
                            if item['name'].lower() in item_name.lower() or item_name.lower() in item['name'].lower():
                                price_match = re.search(r'[\d.]+', price_part)
                                if price_match:
                                    new_price = float(price_match.group())
                                    updated_prices[item['item_id']] = new_price
                                    updated_flags[item['item_id']] = 'UPDATED' in price_part.upper()
                                break
        
        return updated_prices, updated_flags
    
    def calculate_total_cost(self, items, prices):
        total = 0.0
        calculation_details = []
        
        for item in items:
            price = prices.get(item['item_id'], 0.0)
            item_total = item['quantity'] * price
            total += item_total
            calculation_details.append(f"{item['name']}: {item['quantity']} × £{price:.2f} = £{item_total:.2f}")
        
        return total, calculation_details
    
    def verify_calculation(self, calculation_details, total):
        details_str = "\n".join(calculation_details)
        prompt = f"""
        Please verify this shopping cost calculation:
        
        {details_str}
        
        Total: £{total:.2f}
        
        Please confirm whether the numbers are correct with Yes or No. 
        If No, re-calculate the correct total and provide the corrected amount.
        
        Format: [Yes/No] [If No, provide: Correct Total: £X.XX]
        """
        
        response = self.query_perplexity_sonar(prompt, "You are a mathematical verification assistant. Check calculations accurately.")
        
        if response and 'No' in response:
            # Extract corrected total
            total_match = re.search(r'£([\d.]+)', response)
            if total_match:
                try:
                    return float(total_match.group(1))
                except ValueError:
                    pass
        
        return total
    
    def send_email(self, recipient_email, subject, body):
        try:
            msg = MIMEMultipart()
            msg['From'] = self.gmail_email
            msg['To'] = recipient_email
            msg['Subject'] = subject
            
            msg.attach(MIMEText(body, 'plain'))
            
            server = smtplib.SMTP('smtp.gmail.com', 587)
            server.starttls()
            server.login(self.gmail_email, self.gmail_password)
            
            text = msg.as_string()
            server.sendmail(self.gmail_email, recipient_email, text)
            server.quit()
            
            return True
        except Exception as e:
            self.results_text.insert(tk.END, f"Email sending failed: {str(e)}\n")
            return False
    
    def process_shopping_list(self):
        # Clear previous results
        self.results_text.delete(1.0, tk.END)
        self.results_text.insert(tk.END, "Processing your shopping list...\n\n")
        self.root.update()
        
        # Validate inputs
        if not self.validate_inputs():
            return
        
        # Get shopping list
        items = self.get_shopping_list()
        if not items:
            return
        
        postcode = self.postcode_entry.get().strip()
        firstname = self.firstname_entry.get().strip()
        email = self.email_entry.get().strip()
        
        # Step 1: Find closest supermarket
        self.results_text.insert(tk.END, "🔍 Finding closest supermarket...\n")
        self.root.update()
        
        supermarket_info = self.find_closest_supermarket(postcode)
        if not supermarket_info:
            messagebox.showerror("Error", "Could not find supermarket information")
            return
        
        self.results_text.insert(tk.END, f"Supermarket found:\n{supermarket_info}\n\n")
        self.root.update()
        
        # Extract supermarket brand
        supermarket_brand = None
        supermarket_name = ""
        for brand in self.supermarket_brands:
            if brand.lower() in supermarket_info.lower():
                supermarket_brand = brand
                # Extract supermarket name
                name_match = re.search(r'Supermarket Name: (.+)', supermarket_info)
                if name_match:
                    supermarket_name = name_match.group(1).strip()
                break
        
        if not supermarket_brand:
            supermarket_brand = "Tesco"  # Default fallback
            supermarket_name = "Tesco"
        
        # Step 2: Get item prices
        self.results_text.insert(tk.END, "💰 Getting item prices...\n")
        self.root.update()
        
        prices = self.get_item_prices(items, supermarket_brand)
        
        # Step 3: Sense check prices
        self.results_text.insert(tk.END, "✅ Sense checking prices...\n")
        self.root.update()
        
        updated_prices, update_flags = self.sense_check_prices(items, prices)
        
        # Step 4: Calculate total cost
        self.results_text.insert(tk.END, "🧮 Calculating total cost...\n")
        self.root.update()
        
        total_cost, calculation_details = self.calculate_total_cost(items, updated_prices)
        
        # Step 5: Verify calculation
        self.results_text.insert(tk.END, "🔢 Verifying calculations...\n")
        self.root.update()
        
        verified_total = self.verify_calculation(calculation_details, total_cost)
        
        # Extract commute time from supermarket info
        commute_time = "15"  # Default fallback
        time_match = re.search(r'Driving Time: (\d+)', supermarket_info)
        if time_match:
            commute_time = time_match.group(1)
        
        # Display final summary
        summary = f"\n{'='*50}\n"
        summary += f"SHOPPING LIST SUMMARY\n"
        summary += f"{'='*50}\n"
        summary += f"Dear {firstname}, your expected shopping will take {commute_time} minutes, "
        summary += f"and the expected cost would be £{verified_total:.2f}\n\n"
        
        self.results_text.insert(tk.END, summary)
        
        # Step 6: Prepare and send email
        self.results_text.insert(tk.END, "📧 Preparing email...\n")
        self.root.update()
        
        # Create email body
        today_date = datetime.now().strftime("%d %B %Y")
        subject = f"Shopping list – {today_date}"
        
        # Extract address and create Google Maps link
        address_match = re.search(r'Address: (.+)', supermarket_info)
        address = address_match.group(1).strip() if address_match else "Address not found"
        maps_url = f"https://www.google.com/maps/search/{address.replace(' ', '+').replace(',', '%2C')}"
        
        email_body = f"""Dear {firstname},

Thank you for using the smart shopper app today. We recommend you to visit this supermarket: {supermarket_name} and your expected total shopping cost is £{verified_total:.2f}.

The address of your closest supermarket is:
{address}
{maps_url}

Here is the list you asked for:
"""
        
        # Add item details
        website = self.website_mapping.get(supermarket_brand, "www.tesco.com")
        for item in items:
            price = updated_prices.get(item['item_id'], 0.0)
            flag = "Updated in sense-check" if update_flags.get(item['item_id'], False) else "Original price"
            email_body += f"{item['name']}, {item['quantity']}, £{price:.2f}, {flag}, https://{website}\n"
        
        email_body += f"\nTotal Expected: £{verified_total:.2f}\n\nThank you and have a lovely shopping trip!"
        
        # Send email
        self.results_text.insert(tk.END, "📤 Sending email...\n")
        self.root.update()
        
        if self.send_email(email, subject, email_body):
            self.results_text.insert(tk.END, "✅ Email sent successfully!\n")
            self.results_text.insert(tk.END, f"\nFinal_Checked_Email_Message sent to: {email}\n")
        else:
            self.results_text.insert(tk.END, "❌ Failed to send email.\n")
        
        self.results_text.insert(tk.END, "\n🎉 Process completed!\n")

def main():
    root = tk.Tk()
    app = ShoppingListApp(root)
    root.mainloop()

if __name__ == "__main__":
    main()
